In [1]:
import warnings
warnings.simplefilter('ignore')

import gc

import numpy as np
import pandas as pd
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 100)
from tqdm.notebook import tqdm

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GroupKFold, KFold
from sklearn.metrics import mean_squared_error

import lightgbm as lgb

In [2]:
train = pd.read_csv('raw_data/train.csv')
train = train.sort_values(by=['QUEUE_ID', 'DOTTING_TIME']).reset_index(drop=True)

test = pd.read_csv('raw_data/evaluation_public.csv')
test = test.sort_values(by=['ID', 'DOTTING_TIME']).reset_index(drop=True)

sub_sample = pd.read_csv('raw_data/submit_example.csv')

In [3]:
train.head(10)

,QUEUE_ID,CU,STATUS,QUEUE_TYPE,PLATFORM,CPU_USAGE,MEM_USAGE,LAUNCHING_JOB_NUMS,RUNNING_JOB_NUMS,SUCCEED_JOB_NUMS,CANCELLED_JOB_NUMS,FAILED_JOB_NUMS,DOTTING_TIME,RESOURCE_TYPE,DISK_USAGE
0,2,16,available,sql,x86_64,3,54,0,0,0,0,0,1590683100000,vm,20.0
1,2,16,available,sql,x86_64,2,54,0,0,0,0,0,1590683400000,vm,20.0
2,2,16,available,sql,x86_64,7,54,0,0,0,0,0,1590683700000,vm,20.0
3,2,16,available,sql,x86_64,4,54,0,0,0,0,0,1590684000000,vm,20.0
4,2,16,available,sql,x86_64,5,54,0,0,0,0,0,1590684120000,vm,20.0
5,2,16,available,sql,x86_64,3,55,0,0,0,0,0,1590684420000,vm,20.0
6,2,16,available,sql,x86_64,2,54,0,0,0,0,0,1590684720000,vm,20.0
7,2,16,available,sql,x86_64,2,54,0,0,0,0,0,1590685020000,vm,20.0
8,2,16,available,sql,x86_64,5,54,0,0,0,0,0,1590685320000,vm,20.0
9,2,16,available,sql,x86_64,6,54,0,0,0,0,0,1590685620000,vm,20.0


In [4]:
test.head(10)

,ID,QUEUE_ID,CU,STATUS,QUEUE_TYPE,PLATFORM,CPU_USAGE,MEM_USAGE,LAUNCHING_JOB_NUMS,RUNNING_JOB_NUMS,SUCCEED_JOB_NUMS,CANCELLED_JOB_NUMS,FAILED_JOB_NUMS,DOTTING_TIME,RESOURCE_TYPE,DISK_USAGE
0,1,297,16,available,sql,x86_64,60,69,0,5,5,0,0,1662213420000,vm,9
1,1,297,16,available,sql,x86_64,58,69,0,9,4,0,0,1662213720000,vm,9
2,1,297,16,available,sql,x86_64,80,67,0,9,1,0,0,1662214020000,vm,9
3,1,297,16,available,sql,x86_64,100,65,0,7,2,0,1,1662214320000,vm,9
4,1,297,16,available,sql,x86_64,98,67,0,10,3,0,1,1662214620000,vm,9
5,2,85153,64,available,general,x86_64,56,91,0,0,0,0,0,1613655960000,vm,20
6,2,85153,64,available,general,x86_64,48,78,0,1,1,0,0,1613656260000,vm,20
7,2,85153,64,available,general,x86_64,23,35,0,0,0,0,0,1613656560000,vm,20
8,2,85153,64,available,general,x86_64,68,61,0,0,0,0,0,1613656860000,vm,20
9,2,85153,64,available,general,x86_64,38,74,0,0,0,0,0,1613657160000,vm,20


In [5]:
sub_sample.head()

,ID,CPU_USAGE_1,LAUNCHING_JOB_NUMS_1,CPU_USAGE_2,LAUNCHING_JOB_NUMS_2,CPU_USAGE_3,LAUNCHING_JOB_NUMS_3,CPU_USAGE_4,LAUNCHING_JOB_NUMS_4,CPU_USAGE_5,LAUNCHING_JOB_NUMS_5
0,1,0,0,0,0,0,0,0,0,0,0
1,2,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,0
4,5,0,0,0,0,0,0,0,0,0,0


In [6]:
train.shape, test.shape, sub_sample.shape

((501730, 15), (14980, 16), (2996, 11))

In [7]:
# 这些 columns 在 test 只有单一值, 所以直接去掉

del train['STATUS']
del train['PLATFORM']
del train['RESOURCE_TYPE']

del test['STATUS']
del test['PLATFORM']
del test['RESOURCE_TYPE']

In [8]:
# 时间排序好后也没什么用了

del train['DOTTING_TIME']
del test['DOTTING_TIME']

In [9]:
# Label Encoding

le = LabelEncoder()
train['QUEUE_TYPE'] = le.fit_transform(train['QUEUE_TYPE'].astype(str))
test['QUEUE_TYPE'] = le.transform(test['QUEUE_TYPE'].astype(str))

In [10]:
# 只用 CPU_USAGE 和 MEM_USAGE
to_drop_cols = [col for col in train.columns if col.endswith('_JOB_NUMS')] + ['DISK_USAGE']

train.drop(to_drop_cols, axis=1, inplace=True)
test.drop(to_drop_cols, axis=1, inplace=True)

In [11]:
train.head(10)

,QUEUE_ID,CU,QUEUE_TYPE,CPU_USAGE,MEM_USAGE
0,2,16,2,3,54
1,2,16,2,2,54
2,2,16,2,7,54
3,2,16,2,4,54
4,2,16,2,5,54
5,2,16,2,3,55
6,2,16,2,2,54
7,2,16,2,2,54
8,2,16,2,5,54
9,2,16,2,6,54


In [12]:
# t0 t1 t2 t3 t4  ->  t5 t6 t7 t8 t9 
# t1 t2 t3 t4 t5  ->  t6 t7 t8 t9 t10

df_train = pd.DataFrame()

for id_ in tqdm(train.QUEUE_ID.unique()):
    df_tmp = train[train.QUEUE_ID == id_]
    features = list()
    t_cpu = list()
    values = df_tmp.values
    for i, _ in enumerate(values):
        if i + 10 < len(values):
            li_v = list()
            li_v.append(values[i][0])
            li_cpu = list()
            for j in range(5):
                li_v.extend(values[i+j][3:].tolist())
                li_cpu.append(values[i+j+5][3])
            features.append(li_v)
            t_cpu.append(li_cpu)
    df_feat = pd.DataFrame(features)
    df_feat.columns = ['QUEUE_ID', 
                       'CPU_USAGE_1', 'MEM_USAGE_1', 
                       'CPU_USAGE_2', 'MEM_USAGE_2', 
                       'CPU_USAGE_3', 'MEM_USAGE_3', 
                       'CPU_USAGE_4', 'MEM_USAGE_4', 
                       'CPU_USAGE_5', 'MEM_USAGE_5', 
                      ]
    df_cpu = pd.DataFrame(t_cpu)
    df_cpu.columns = ['cpu_1', 'cpu_2', 'cpu_3', 'cpu_4', 'cpu_5']
    df = pd.concat([df_feat, df_cpu], axis=1)
    print(f'QUEUE_ID: {id_}, lines: {df.shape[0]}')
    df_train = df_train.append(df)

QUEUE_ID: 2, lines: 19245
QUEUE_ID: 3, lines: 19247
QUEUE_ID: 4, lines: 19247
QUEUE_ID: 26, lines: 10397
QUEUE_ID: 27, lines: 10516
QUEUE_ID: 36, lines: 3232
QUEUE_ID: 233, lines: 2089
QUEUE_ID: 281, lines: 10355
QUEUE_ID: 287, lines: 6866
QUEUE_ID: 291, lines: 8874
QUEUE_ID: 293, lines: 8845
QUEUE_ID: 297, lines: 21174
QUEUE_ID: 298, lines: 20371
QUEUE_ID: 20889, lines: 13995
QUEUE_ID: 21487, lines: 28921
QUEUE_ID: 21671, lines: 28085
QUEUE_ID: 21673, lines: 19716
QUEUE_ID: 21825, lines: 19713
QUEUE_ID: 81221, lines: 19771
QUEUE_ID: 82695, lines: 19716
QUEUE_ID: 82697, lines: 10632
QUEUE_ID: 82929, lines: 10189
QUEUE_ID: 83109, lines: 8948
QUEUE_ID: 83609, lines: 2110
QUEUE_ID: 84151, lines: 11847
QUEUE_ID: 84387, lines: 17510
QUEUE_ID: 84907, lines: 6485
QUEUE_ID: 85101, lines: 6608
QUEUE_ID: 85153, lines: 14343
QUEUE_ID: 85265, lines: 13506
QUEUE_ID: 85267, lines: 13072
QUEUE_ID: 85617, lines: 2543
QUEUE_ID: 85619, lines: 9987
QUEUE_ID: 85693, lines: 10824
QUEUE_ID: 85731, lines: 85

In [13]:
df_test = pd.DataFrame()

for id_ in tqdm(test.QUEUE_ID.unique()):
    df_tmp = test[test.QUEUE_ID == id_]
    features = list()
    values = df_tmp.values
    for i, _ in enumerate(values):
        if i % 5 == 0:
            li_v = list()
            li_v.append(values[i][0])
            li_v.append(values[i][1])
            for j in range(5):
                li_v.extend(values[i+j][4:].tolist())
            features.append(li_v)
    df_feat = pd.DataFrame(features)
    df_feat.columns = ['ID', 'QUEUE_ID', 
                       'CPU_USAGE_1', 'MEM_USAGE_1', 
                       'CPU_USAGE_2', 'MEM_USAGE_2', 
                       'CPU_USAGE_3', 'MEM_USAGE_3', 
                       'CPU_USAGE_4', 'MEM_USAGE_4', 
                       'CPU_USAGE_5', 'MEM_USAGE_5', 
                      ]
    df = df_feat.copy()
    print(f'QUEUE_ID: {id_}, lines: {df.shape[0]}')
    df_test = df_test.append(df)

QUEUE_ID: 297, lines: 1142
QUEUE_ID: 85153, lines: 390
QUEUE_ID: 291, lines: 57
QUEUE_ID: 21487, lines: 447
QUEUE_ID: 85265, lines: 19
QUEUE_ID: 4, lines: 151
QUEUE_ID: 2, lines: 151
QUEUE_ID: 81221, lines: 52
QUEUE_ID: 287, lines: 53
QUEUE_ID: 85693, lines: 24
QUEUE_ID: 3, lines: 156
QUEUE_ID: 293, lines: 51
QUEUE_ID: 36, lines: 33
QUEUE_ID: 26, lines: 58
QUEUE_ID: 281, lines: 100
QUEUE_ID: 83609, lines: 3
QUEUE_ID: 21671, lines: 29
QUEUE_ID: 27, lines: 57
QUEUE_ID: 233, lines: 11
QUEUE_ID: 85101, lines: 2
QUEUE_ID: 85933, lines: 4
QUEUE_ID: 21673, lines: 4
QUEUE_ID: 298, lines: 2



In [14]:
# 行内统计特征

df_train['cpu_mean'] = df_train[[f'CPU_USAGE_{i}' for i in range(1,6)]].mean(axis=1)
df_train['cpu_std'] = df_train[[f'CPU_USAGE_{i}' for i in range(1,6)]].std(axis=1)
df_train['cpu_diff'] = df_train['CPU_USAGE_5'] - df_train['CPU_USAGE_1']
df_train['cpu_max'] = df_train[[f'CPU_USAGE_{i}' for i in range(1,6)]].max(axis=1)
df_train['mem_mean'] = df_train[[f'MEM_USAGE_{i}' for i in range(1,6)]].mean(axis=1)
df_train['mem_std'] = df_train[[f'MEM_USAGE_{i}' for i in range(1,6)]].std(axis=1)
df_train['mem_max'] = df_train[[f'MEM_USAGE_{i}' for i in range(1,6)]].max(axis=1)

df_test['cpu_mean'] = df_test[[f'CPU_USAGE_{i}' for i in range(1,6)]].mean(axis=1)
df_test['cpu_std'] = df_test[[f'CPU_USAGE_{i}' for i in range(1,6)]].std(axis=1)
df_test['cpu_diff'] = df_test['CPU_USAGE_5'] - df_test['CPU_USAGE_1']
df_test['cpu_max'] = df_test[[f'CPU_USAGE_{i}' for i in range(1,6)]].max(axis=1)
df_test['mem_mean'] = df_test[[f'MEM_USAGE_{i}' for i in range(1,6)]].mean(axis=1)
df_test['mem_std'] = df_test[[f'MEM_USAGE_{i}' for i in range(1,6)]].std(axis=1)
df_test['mem_max'] = df_test[[f'MEM_USAGE_{i}' for i in range(1,6)]].max(axis=1)

In [15]:
print(df_train.shape)
df_train.head()

(501300, 23)


,QUEUE_ID,CPU_USAGE_1,MEM_USAGE_1,CPU_USAGE_2,MEM_USAGE_2,CPU_USAGE_3,MEM_USAGE_3,CPU_USAGE_4,MEM_USAGE_4,CPU_USAGE_5,MEM_USAGE_5,cpu_1,cpu_2,cpu_3,cpu_4,cpu_5,cpu_mean,cpu_std,cpu_diff,cpu_max,mem_mean,mem_std,mem_max
0,2,3,54,2,54,7,54,4,54,5,54,3,2,2,5,6,4.2,1.923538,2,7,54.0,0.000000,54
1,2,2,54,7,54,4,54,5,54,3,55,2,2,5,6,2,4.2,1.923538,1,7,54.2,0.447214,55
2,2,7,54,4,54,5,54,3,55,2,54,2,5,6,2,3,4.2,1.923538,-5,7,54.2,0.447214,55
3,2,4,54,5,54,3,55,2,54,2,54,5,6,2,3,10,3.2,1.303840,-2,5,54.2,0.447214,55
4,2,5,54,3,55,2,54,2,54,5,54,6,2,3,10,6,3.4,1.516575,0,5,54.2,0.447214,55


In [16]:
print(df_test.shape)
df_test.head()

(2996, 19)


,ID,QUEUE_ID,CPU_USAGE_1,MEM_USAGE_1,CPU_USAGE_2,MEM_USAGE_2,CPU_USAGE_3,MEM_USAGE_3,CPU_USAGE_4,MEM_USAGE_4,CPU_USAGE_5,MEM_USAGE_5,cpu_mean,cpu_std,cpu_diff,cpu_max,mem_mean,mem_std,mem_max
0,1,297,60,69,58,69,80,67,100,65,98,67,79.2,20.029978,38,100,67.4,1.673320,69
1,3,297,2,41,5,42,3,43,80,68,90,82,36.0,44.883182,88,90,55.2,18.753666,82
2,4,297,3,34,37,46,90,71,64,72,68,68,52.4,33.426038,65,90,58.2,17.210462,72
3,7,297,32,51,84,78,52,80,31,80,32,77,46.2,22.895414,0,84,73.2,12.477981,80
4,11,297,50,67,69,64,2,64,2,64,2,47,25.0,32.202484,-48,69,61.2,8.043631,67


In [17]:
def run_lgb_qid(df_train, df_test, target, qid):
    
    feature_names = list(
        filter(lambda x: x not in ['QUEUE_ID', 'CU', 'QUEUE_TYPE'] + [f'cpu_{i}' for i in range(1,6)], 
               df_train.columns))
    
    # 提取 QUEUE_ID 对应的数据集
    df_train = df_train[df_train.QUEUE_ID == qid]
    df_test = df_test[df_test.QUEUE_ID == qid]
    
    print(f"QUEUE_ID:{qid}, target:{target}, train:{len(df_train)}, test:{len(df_test)}")
    
    model = lgb.LGBMRegressor(num_leaves=20,
                              max_depth=4,
                              learning_rate=0.08,
                              n_estimators=10000,
                              subsample=0.9,
                              feature_fraction=0.8,
                              reg_alpha=0.6,
                              reg_lambda=1.2,
                              random_state=42)
    oof = []
    prediction = df_test[['ID', 'QUEUE_ID']]
    prediction[target] = 0
    
    kfold = KFold(n_splits=5, random_state=42)
    for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(df_train, df_train[target])):
        
        X_train = df_train.iloc[trn_idx][feature_names]
        Y_train = df_train.iloc[trn_idx][target]
        X_val = df_train.iloc[val_idx][feature_names]
        Y_val = df_train.iloc[val_idx][target]
        
        lgb_model = model.fit(X_train, 
                              Y_train,
                              eval_names=['train', 'valid'],
                              eval_set=[(X_train, Y_train), (X_val, Y_val)],
                              verbose=0,
                              eval_metric='mse',
                              early_stopping_rounds=20)
        
        pred_val = lgb_model.predict(X_val, num_iteration=lgb_model.best_iteration_)
        df_oof = df_train.iloc[val_idx][[target, 'QUEUE_ID']].copy()
        df_oof['pred'] = pred_val
        oof.append(df_oof)
        
        pred_test = lgb_model.predict(df_test[feature_names], num_iteration=lgb_model.best_iteration_)
        prediction[target] += pred_test / kfold.n_splits
        
        del lgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
        gc.collect()
        
    df_oof = pd.concat(oof)
    score = mean_squared_error(df_oof[target], df_oof['pred'])
    print('MSE:', score)

    return prediction, score

In [18]:
predictions = list()
scores = list()

for qid in tqdm(test.QUEUE_ID.unique()):    
    df = pd.DataFrame()
    for t in [f'cpu_{i}' for i in range(1,6)]:
        prediction, score = run_lgb_qid(df_train, df_test, target=t, qid=qid)
        if t == 'cpu_1':
            df = prediction.copy()
        else:
            df = pd.merge(df, prediction, on=['ID', 'QUEUE_ID'], how='left')            
        scores.append(score)

    predictions.append(df)

QUEUE_ID:297, target:cpu_1, train:21174, test:1142
MSE: 149.32876070057813
QUEUE_ID:297, target:cpu_2, train:21174, test:1142
MSE: 230.2215851475953
QUEUE_ID:297, target:cpu_3, train:21174, test:1142
MSE: 234.30335117861307
QUEUE_ID:297, target:cpu_4, train:21174, test:1142
MSE: 195.3317566185578
QUEUE_ID:297, target:cpu_5, train:21174, test:1142
MSE: 144.0756453053243
QUEUE_ID:85153, target:cpu_1, train:14343, test:390
MSE: 166.59306316840988
QUEUE_ID:85153, target:cpu_2, train:14343, test:390
MSE: 190.96021024410138
QUEUE_ID:85153, target:cpu_3, train:14343, test:390
MSE: 196.66236632719
QUEUE_ID:85153, target:cpu_4, train:14343, test:390
MSE: 200.01051626902412
QUEUE_ID:85153, target:cpu_5, train:14343, test:390
MSE: 202.92113111605087
QUEUE_ID:291, target:cpu_1, train:8874, test:57
MSE: 82.34531737776904
QUEUE_ID:291, target:cpu_2, train:8874, test:57
MSE: 178.4386014114649
QUEUE_ID:291, target:cpu_3, train:8874, test:57
MSE: 238.13296273801464
QUEUE_ID:291, target:cpu_4, train:887

MSE: 7.906314894555713
QUEUE_ID:298, target:cpu_4, train:20371, test:2
MSE: 8.096041288743692
QUEUE_ID:298, target:cpu_5, train:20371, test:2
MSE: 8.177420585373383



In [19]:
print('mean MSE score: ', np.mean(scores))

mean MSE score:  55.86956396891888


In [20]:
sub = pd.concat(predictions)

sub = sub.sort_values(by='ID').reset_index(drop=True)
sub.drop(['QUEUE_ID'], axis=1, inplace=True)
sub.columns = ['ID'] + [f'CPU_USAGE_{i}' for i in range(1,6)]

# 全置 0 都比训练出来的结果好
for col in [f'LAUNCHING_JOB_NUMS_{i}' for i in range(1,6)]:
    sub[col] = 0
    
sub = sub[['ID',
           'CPU_USAGE_1', 'LAUNCHING_JOB_NUMS_1', 
           'CPU_USAGE_2', 'LAUNCHING_JOB_NUMS_2', 
           'CPU_USAGE_3', 'LAUNCHING_JOB_NUMS_3', 
           'CPU_USAGE_4', 'LAUNCHING_JOB_NUMS_4', 
           'CPU_USAGE_5', 'LAUNCHING_JOB_NUMS_5']]

print(sub.shape)
sub.head()

(2996, 11)


,ID,CPU_USAGE_1,LAUNCHING_JOB_NUMS_1,CPU_USAGE_2,LAUNCHING_JOB_NUMS_2,CPU_USAGE_3,LAUNCHING_JOB_NUMS_3,CPU_USAGE_4,LAUNCHING_JOB_NUMS_4,CPU_USAGE_5,LAUNCHING_JOB_NUMS_5
0,1,90.795396,0,86.232249,0,85.889177,0,93.051595,0,92.070281,0
1,2,38.108620,0,32.646857,0,35.574923,0,42.289413,0,27.725214,0
2,3,38.876593,0,57.453038,0,13.975760,0,9.631157,0,1.637484,0
3,4,23.553786,0,16.824533,0,6.838082,0,4.016654,0,3.538453,0
4,5,3.203495,0,6.925173,0,10.519199,0,10.891437,0,9.141628,0


In [21]:
# 注意: 提交要求预测结果需为非负整数, 包括 ID 也需要是整数

sub['ID'] = sub['ID'].astype(int)

for col in [i for i in sub.columns if i != 'ID']:
    sub[col] = sub[col].apply(np.floor)
    sub[col] = sub[col].apply(lambda x: 0 if x<0 else x)
    sub[col] = sub[col].astype(int)
    
sub.head(10)

,ID,CPU_USAGE_1,LAUNCHING_JOB_NUMS_1,CPU_USAGE_2,LAUNCHING_JOB_NUMS_2,CPU_USAGE_3,LAUNCHING_JOB_NUMS_3,CPU_USAGE_4,LAUNCHING_JOB_NUMS_4,CPU_USAGE_5,LAUNCHING_JOB_NUMS_5
0,1,90,0,86,0,85,0,93,0,92,0
1,2,38,0,32,0,35,0,42,0,27,0
2,3,38,0,57,0,13,0,9,0,1,0
3,4,23,0,16,0,6,0,4,0,3,0
4,5,3,0,6,0,10,0,10,0,9,0
5,6,13,0,14,0,13,0,11,0,10,0
6,7,10,0,8,0,6,0,2,0,3,0
7,8,0,0,0,0,0,0,0,0,0,0
8,9,3,0,4,0,3,0,3,0,3,0
9,10,15,0,10,0,11,0,10,0,11,0


In [22]:
sub.shape, sub_sample.shape

((2996, 11), (2996, 11))

In [23]:
sub.to_csv('baseline.csv', index=False)